In [1]:
import wandb
from pathlib import Path
import os
from tqdm.auto import tqdm
from PIL import Image
import numpy as np

In [2]:
os.environ['WANDB_NOTEBOOK_NAME']='EDA.ipynb'
os.environ['WANDB_API_KEY']='be69ec2b537cb972b0106fabd867f0dc68c4468a'

In [3]:
data_path=Path('COD10K-v2')
train_path=data_path / 'Train'
test_path= data_path / 'Test'

train_images_path= train_path / 'Images/Image'
train_labels_path= train_path / 'GT_Objects/GT_Object'

In [4]:
train_images_list=list(train_images_path.glob('*.jpg'))
print(train_images_list[0:5])
train_labels_list=list(train_labels_path.glob('*.png'))
print(train_labels_list[0:5])

[WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-1.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-3.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-7.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-8.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-9.jpg')]
[WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-1.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-3.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-7.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-8.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-9.png')]


In [5]:
super_classes_list=[ i.stem.split('-')[3] for i in train_images_list]
super_classes_list=[*set(super_classes_list)]
print(super_classes_list)

['Other', 'Terrestial', 'Amphibian', 'Background', 'Flying', 'Terrestrial', 'Aquatic']


In [6]:
index=5
sub_classes_list=[]
for i in train_images_list:
    split_path=i.stem.split('-')
    count=len(split_path)
    if (index<count):
        sub_classes_list.append(split_path[index])
sub_classes_list=[*set(sub_classes_list)]
print(sub_classes_list)


['Bee', 'Other', 'Lion', 'StarFish', 'Ocean', 'Crab', 'Fish', 'Sky', 'Frog', 'Pipefish', 'Ant', 'Chameleon', 'Flounder', 'Frogmouth', 'Vegetation', 'Bat', 'Lizard', 'BatFish', 'Kangaroo', 'Owl', 'Toad', 'Butterfly', 'Worm', 'Cheetah', 'Human', 'Sheep', 'Pagurian', 'Slug', 'Stingaree', 'Deer', 'LeafySeaDragon', 'Dog', 'Turtle', 'Grasshopper', 'Moth', 'CrocodileFish', 'Monkey', 'Tiger', 'Cicada', 'StickInsect', 'Bittern', 'FrogFish', 'Heron', 'Indoor', 'Sand', 'Octopus', 'Rabbit', 'Reccoon', 'Beetle', 'Giraffe', 'Leopard', 'Katydid', 'ClownFish', 'Crocodile', 'Sciuridae', 'Spider', 'Dragonfly', 'Bird', 'ScorpionFish', 'Duck', 'Centipede', 'Grouse', 'Mockingbird', 'Caterpillar', 'Bug', 'Snake', 'GhostPipefish', 'Mantis', 'SeaHorse', 'Gecko', 'Cat', 'Wolf', 'Owlfly', 'Shrimp']


In [7]:
table = wandb.Table(columns=["File_Name", "Images", "Split"] )

In [9]:
for i, image_file in enumerate(tqdm(train_images_list)):
    image=Image.open(image_file)
    label_name= train_labels_path / (image_file.stem)
    label_name=str(label_name)+'.png'
    label=np.array(Image.open(label_name))
    split_path=image_file.stem.split('-')
    super_class=split_path[3]
    count=len(split_path)
    if(5< count):
        sub_class=split_path[5]
    else:
        sub_class='None'
    table.add_data(str(image_file),
                    wandb.Image(
                        image,
                        masks={
                            "predictions" : {
                                "mask_data" : label

                            }
                        }

                    ),
                    'Train'
            
                    
    )

  0%|          | 0/6000 [00:00<?, ?it/s]

In [8]:
run = wandb.init(project='Camouflage-Object-Detection', entity='sathishkumartheta', job_type="upload")




wandb: Currently logged in as: sathishkumartheta. Use `wandb login --relogin` to force relogin


In [10]:
raw_data_at = wandb.Artifact('COD10K-v2', type="raw_data")
raw_data_at.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='WvxNzSCf6wjohNs+1w2cbw==', ref=None, birth_artifact_id=None, size=3703292, extra={}, local_path='C:\\Users\\Sathishkumar\\AppData\\Local\\wandb\\wandb\\artifacts\\staging\\tmpnunpp0ln')

In [11]:

run.log_artifact(raw_data_at)
run.finish()